In [1]:
!pip install datasets llama-index tenacity sklearn nest_asyncio tiktoken seaborn

In [2]:
import datasets
import tiktoken
from functools import partial
from datasets import load_dataset
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from concurrent.futures import ThreadPoolExecutor
import asyncio
from llama_index.llms import ChatMessage, OpenAI
import openai

import nest_asyncio
nest_asyncio.apply()

pd.set_option("display.max_colwidth", 1000)

In [3]:
train_data = load_dataset("ms_marco", 'v2.1', split="train")
test_data = load_dataset("ms_marco", 'v2.1', split="test")
validation_data = load_dataset("ms_marco", 'v2.1', split="validation")

In [4]:
train_df, test_df, validation_df = train_data.to_pandas(), test_data.to_pandas(), validation_data.to_pandas()

In [5]:
train_df.head(1)

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[The immediate impact of the success of the manhattan project was the only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.],"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'passage_text': ['The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.', 'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.', 'Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.', 'The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period of the project fr...",)what was the immediate impact of the success of the manhattan project?,1185869,DESCRIPTION,[]


In [ ]:
# Separating is_selected and passages for better training
# Expand the 'passages' column into separate columns
def process_row(row):
    passages = pd.DataFrame({
        "is_selected": row['passages']['is_selected'],
        "passage_text": row['passages']['passage_text']
    })
    
    # Repeat other columns for the length of passages
    for col in ['answers', 'query', 'query_id', 'query_type', 'wellFormedAnswers']:
        passages[col] = [row[col]] * len(passages)
    
    return passages


# Apply the function to each row
dfs = train_df.apply(process_row, axis=1)

# Concatenate the resulting DataFrames and reset index
result = pd.concat(dfs.values).reset_index(drop=True)

# Convert 'answers' column list values to string values
result['answers'] = result['answers'].str[0]

In [ ]:
result.head()